In [ ]:
!pip install bertopic==0.14.1
!pip install --upgrade joblib==1.1.0
!pip install --upgrade tbb
!pip install stanza==1.4.0
!pip install jsonlines
!pip install jiwer==2.2.1
!pip install spacy[transformers]
!python -m spacy download en_core_web_sm
# !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
import os, sys
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import random
import transformers
transformers.set_seed(0)
random.seed(0)

import nltk
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize

import ast
import itertools
from tqdm import tqdm
tqdm.pandas()
import itertools
import json



import re
import string
import jiwer

import gensim
import pprint
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from sentence_transformers import SentenceTransformer, CrossEncoder, util, models
cross_encoder = CrossEncoder('cross-encoder/stsb-distilroberta-base',device=device)
entail_encoder = CrossEncoder('cross-encoder/nli-distilroberta-base',device=device)

# sentence-transformers/all-mpnet-base-v2
word_embedding_model = SentenceTransformer('flax-sentence-embeddings/stackoverflow_mpnet-base',device=device)


import math
from scipy.spatial import distance
from scipy.signal import find_peaks
from scipy.stats import chi2_contingency

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.patches as mpatches
plt.rcParams["figure.figsize"] = (11.7,8.27)
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})

main_arguments = ['ARG' + str(idx) for idx in range(6)]

from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!nvidia-smi

In [ ]:
def process_(text):
  text = text.replace("\r\n"," ").replace("\n"," ")
  text = re.sub('podling', 'project', text,flags=re.IGNORECASE)
  regex = r'\b(\w+)(?:\W+\1\b)+'
  text = re.sub(regex, r'\1', text, flags=re.IGNORECASE)
  #remove URL
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  return text.lower()

In [ ]:
# only topic rules

rules_data = pd.read_csv('asf_rules.csv')
rules = rules_data['rules'].tolist();print(len(rules))
topic_model = BERTopic.load('topic_model',embedding_model=word_embedding_model)
print("cluster density:",topic_model.hdbscan_model.relative_validity_)


freq = topic_model.get_topic_info();print('No of topics',freq.shape[0])
asf_tops,_ = topic_model.transform(rules)
top_map = pd.DataFrame(columns=['rule','label'])
top_map['rules'],top_map['label'] = rules,asf_tops
top_map['topic'] = top_map['label'].apply(lambda x : freq[freq['Topic'] == x].iloc[0]['Name'])
top_map.sort_values(by=['label'],inplace=True)
top_map.to_csv('cluster_rules_outlier.csv',index=False)
top_map = top_map[top_map['label'] > -1];top_map.to_csv('cluster_rules.csv')
print('ASF Topics: ', len(top_map['label'].unique()))
freq.to_csv('freq.csv',index=False)
print(topic_model.hdbscan_model.min_cluster_size,topic_model.hdbscan_model.min_samples,topic_model.umap_model.n_components,topic_model.umap_model.n_neighbors)

In [ ]:
#generate topic predictions using tuned model
topic_model = BERTopic.load('topic_model',embedding_model=word_embedding_model)
print("cluster density:",topic_model.hdbscan_model.relative_validity_)
all_data = pd.read_csv('srl_clauses_processed_en.csv');all_data.shape[0]

all_data['parsed_clauses'] = all_data['parsed_clauses'].apply(lambda x : process_(x))
all_data['parsed_clauses'] = all_data['parsed_clauses'].apply(lambda x : x if len(x.strip()) > 2 else np.nan); all_data.dropna(subset=['parsed_clauses'],inplace=True)

final_data = pd.DataFrame(columns = list(all_data.columns) + ['group_id'])
print('All data: ', all_data.shape[0])
for df in np.array_split(all_data,20):
  print(df.shape[0])
  df["group_id"] = topic_model.transform(df['parsed_clauses'].tolist())[0]
  final_data = final_data.append(df);print(final_data.shape[0])

assert all_data.shape[0] == final_data.shape[0]

#keep only parsed activities related to governance topics

final_data = final_data[(final_data['group_id'].isin(asf_tops)) & (final_data['group_id'] > -1)]
freq = topic_model.get_topic_info()
final_data["topic"] = final_data['group_id'].apply(lambda x : freq[freq['Topic'] == x].iloc[0]['Name'])
final_data.to_csv('governed_activities.csv'); print(final_data.shape[0])


